# TAXII 2.0 Tutorial

Connect to the UberTAXII server, list API roots, and pull some STIX data.

In [ ]:
# A "Server" object pulls general information about the server, and gives you a list of API roots.
# Connect to it by providing the URL, username, and password.

from taxii2client import Server
server = Server('https://ubertaxii.com/taxii/', user='ubertaxii', password='certain_players_want_a_pony')

In [ ]:
# Print some basic information, available without making additional requests.

print("Title:\t\t" + server.title)
print("Description:\t" + server.description)

In [ ]:
# Iterate through the API roots. Note that addtional requests are made automatically to get the title and description.

for api_root in server.api_roots:
    print(api_root.title)
    print(api_root.description)
    print(api_root.url)
    print("\n")

In [ ]:
# Get the list of collections for an API root (using MITRE's)

mitre_api_root = server.api_roots[3]
for collection in mitre_api_root.collections:
    print(collection.title)

# STIX 2.0 Tutorial

Starting with our reference to a TAXII server, do some stuff with STIX data

In [ ]:
from stix2 import TAXIICollectionSource, Filter

taxii_source = TAXIICollectionSource(mitre_api_root.collections[0])

In [ ]:
results = taxii_source.query(Filter("type", "=", "intrusion-set"))
print("Got {} results".format(len(results)))

In [ ]:
apt_xxx = results[36]
print(apt_xxx.name) # Note that it's a python-stix2 object already, no need to parse the JSON

In [ ]:
# What malware and tools does that intrusion set use?

# Find relationships where the source is that intrusion set
all_relationships = taxii_source.query(Filter("type", "=", "relationship"))
relationships = [r for r in all_relationships if r.source_ref == apt_xxx.id]
print("Found {} relationships".format(len(relationships)))

In [ ]:
for rel in relationships:
    target = taxii_source.get(rel.target_ref)
    print("{} {} {} ({})".format(apt_xxx.name, rel.relationship_type, target.name, target.type))

# Gotcha! Note that this does not optimize the number of calls.
# You still need to know when a TAXII request is going to be made and when the result is available locally.
# Also, watch for n+1 queries (like here) and try to optimize if possible

## Make it Easy: Environments
Use environments to add default sources, sinks, etc.

In [ ]:
from stix2 import Environment, MemoryStore

taxiisource = TAXIICollectionSource(mitre_api_root.collections[0])
env = Environment(source=taxiisource)

In [ ]:
env.get(apt_xxx.id).name

In [ ]:
# Try the other direction
rels = env.query(Filter("source_ref", "=", apt_xxx.id))
print(len(rels))

# Note the delay: not all filters happen server-side,
# it sometimes transparently just pulls all data and filters at the client

In [ ]:
malware = env.get(rels[1].target_ref)
malware.name

## Sharing back: Creating STIX

You can also use the enviornment to set defaults so you don't have to duplicate adding standard markings, etc.

In [ ]:
from stix2 import Indicator, Relationship, ObjectFactory, CompositeDataSource

factory = ObjectFactory(created_by_ref="identity--311b2d2d-f010-5473-83ec-1edf84858f4c")
source = CompositeDataSource()
memorystore = MemoryStore()
source.add_data_sources([memorystore.source, taxiisource])
env2 = Environment(factory=factory, source=source, sink=memorystore)

ind = factory.create(Indicator,
                     labels=["malicious-activity"],
                     pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']")
rel = factory.create(Relationship, source_ref=ind.id, target_ref=malware.id, relationship_type="indicates")
print(rel)

In [ ]:
# Push to our memory data source
env2.add(ind)
env2.add(rel)

In [ ]:
# Is it still there?
print(env2.get(rel.id))

In [ ]:
# setup writable API root
from stix2 import TAXIICollectionSink

scratch_api_root = server.api_roots[4]
for collection in scratch_api_root.collections:
    print(collection.title)
taxii_sink = TAXIICollectionSink(scratch_api_root.collections[0])
env3 = Environment(factory=factory, source=source, sink=taxii_sink)

In [ ]:
# Push to our memory data source
env3.add(ind)
env3.add(rel)

In [ ]:
# Is it still there?
print(env3.get(rel.id))

# NOTE: This is hitting the TAXII collection we just wrote to...

# STIX 2.1 preview

Let's try some fun things...

In [ ]:
# Jupyter notebook thing...restart our kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
# setup our TAXII environment...
from taxii2client import Collection
from stix2 import TAXIICollectionSource

scratch_collection = Collection("https://ubertaxii.com/taxii/scratch/collections/4a26ee4a-924c-4d5e-8519-5d87efe7f6a8/", user="ubertaxii", password="certain_players_want_a_pony")


# STIX 2.1 Preview

## i18n

Let's pretend we're an ISAC publishing an indicator in several different languages.

In [ ]:
from stix2 import Indicator, LanguageContent, Identity, Campaign, ThreatActor, Relationship, Opinion, ObjectFactory, CompositeDataSource, MemoryStore

# create an indicator
indicator = Indicator(name="File hash for malware variant", labels=["malicious-activity"], 
                      pattern="[file:hashes.md5 = 'd41d8cd98f00b204e9800998ecf8427e']",
                     created_by_ref="identity--3d368fac-7e96-4c66-b46c-8c763194a523",
                     lang="en")

# create a language content object for our translations
indicator_trans = LanguageContent(object_ref=indicator.id, object_modified=indicator.modified, 
    contents={
        "fr": {"name": "Fichier hash pour la variante du malware"},
        "nl": {"name": "Bestandshash voor malware-variant"},
        "de": {"name": "Dateihash für Malware-Variante"},
        "zh": {"name": "文件散列用於惡意軟件變體"},
        "ru": {"name": "Хэш файла для варианта вредоносного ПО"},
        "ja": {"name": "マルウェアバリアントのファイルハッシュ"},
        "eo": {"name": "Dosiero hash por malware varianto"},
        "ar": {"name": "تجزئة ملف لبرامج ضارة مختلفة"}
        },
    created_by_ref="identity--3d368fac-7e96-4c66-b46c-8c763194a523")

print(indicator_trans)


# let's create some example producer identities
isac_producer = Identity(name="CarnISAC", identity_class="organization", 
                         id="identity--3d368fac-7e96-4c66-b46c-8c763194a523", confidence=95)
threat_actor = ThreatActor(name="Doctor Evil", labels="terrorist", 
                           created_by_ref="identity--3d368fac-7e96-4c66-b46c-8c763194a523", confidence=95)
campaign = Campaign(name="Blow-up the moon", created_by_ref="identity--3d368fac-7e96-4c66-b46c-8c763194a523", 
                    confidence=80)
rel1 = Relationship(source_ref=campaign.id, target_ref=threat_actor.id, relationship_type="attributed-to",
                  created_by_ref="identity--3d368fac-7e96-4c66-b46c-8c763194a523",
                   confidence=75)
rel2 = Relationship(source_ref=indicator.id, target_ref=threat_actor.id, relationship_type="indicates",
                  created_by_ref="identity--3d368fac-7e96-4c66-b46c-8c763194a523",
                   confidence=70)

print(threat_actor, campaign, rel1)

company_foo = Identity(name="Foo, Inc.", identity_class="organization", 
                       id="identity--dbc4c981-60c4-45bd-a988-b56801e484f5")
company_bar = Identity(name="Bar, Inc.", identity_class="organization", 
                       id="identity--26eea235-040f-46e8-b6ef-5592c88da7ed")
company_baz = Identity(name="Baz, Inc.", identity_class="organization", 
                       id="identity--31e8b302-7654-4fa5-8b1e-608222dea432")


company_foo_opinion = Opinion(created_by_ref="identity--dbc4c981-60c4-45bd-a988-b56801e484f5", object_refs=rel1.id, 
                              opinion="strongly-disagree")
company_bar_opinion = Opinion(created_by_ref="identity--26eea235-040f-46e8-b6ef-5592c88da7ed", object_refs=rel1.id, 
                              opinion="agree")
company_baz_opinion = Opinion(created_by_ref="identity--31e8b302-7654-4fa5-8b1e-608222dea432", object_refs=rel1.id, 
                              opinion="disagree")

In [ ]:
# let's analyze the community's assessment of CarnISAC's campaign attribution...

opinion_map = {
    "strongly-disagree": 1,
    "disagree": 2,
    "neutral": 3,
    "agree": 4,
    "strongly-agree": 5
}

opinion_reverse_map = {v: k for k, v in opinion_map.items()}


opinions = [opinion_map[company_foo_opinion.opinion], opinion_map[company_bar_opinion.opinion], opinion_map[company_foo_opinion.opinion]]
opinion_average = sum(opinions)/(len(opinions))

print("CarnISAC's campaign attribution confidence: %i" % (rel1.confidence))
print("Average trust-group opinion of campaign attribution: %s" % (opinion_reverse_map[opinion_average]))

In [ ]:
from stix2.utils import new_version

def trans_lang(lang_content_obj, obj, lang):
    new_props = {}
    if lang in lang_content_obj.contents:
        for i in lang_content_obj.contents[lang]:
            if i in obj:
                new_props[i] = lang_content_obj.contents[lang][i]
    new_props["lang"] = lang
    trans_obj = new_version(obj, **new_props)
    return(trans_obj)

new_obj = trans_lang(indicator_trans, indicator, 'fr')
print(repr(new_obj))